In [231]:
from IPython.display import clear_output
import time
import numpy as np

pieces = {0: ' ', 1: '|', 2: 'X', 3: '-', 4: 'o'}
# x, y, s means piece s at x,y
# -1, 0, s means score s

def drawer(out):
    max_x, max_y = max(out[::3]), max(out[1::3])
    print(max_x, max_y)
    score = 0
    grid = np.ones((max_y+1, max_x+1)) * -1#[[-1] * (max_x+1)] * (max_y+1)
    for i in range(0, len(out), 3):
        if out[i] == -1 or out[i+1] == -1:
            score = out[i+2]
        else:
            grid[out[i+1],out[i]] = out[i+2]
    for line in grid:
        print(line)
    output = [''.join(map(lambda x: pieces[x], line)) for line in grid] + [f"Score: {score}"]
    for line in output:
        print(line)


In [232]:
import advent
data = advent.get_intcode(13)

code = data.copy()
code[0] = 2 # insert coin
_, out = run(code, IO())

22 18
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1.]
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1.]
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1.]
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1.]
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1.]
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1.]
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1.]
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1.]
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1.]
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1.]
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.

KeyError: -1.0

In [215]:
def put(code, op, pointer, relative, offset, value):
    mode = op % (10**(2+offset)) // (10**(1+offset))
    if mode == 0: # parameter mode
        ix = code[pointer+offset]
    elif mode == 2: # relative mode
        ix = code[pointer+offset] + relative
    else: raise ValueError(f"{op}, {pointer}, {offset} not allowed")

    if ix >= len(code): # Make sure we allocate enough memory
        code += [0] * (ix - len(code) + 1)
    code[ix] = value

def get(op, code, pointer, relative, offset):
    # e.g. get(1002, code, 2) -> code[p] (immediate mode)
    # e.g. get(2, code, 2) -> code[code[p]] (parameter mode)
    mode = op % (10**(2+offset)) // (10**(1+offset))
    if mode == 0: # parameter mode
        if pointer+offset >= len(code): return 0 if code[0] >= len(code) else code[0]
        if code[pointer+offset] >= len(code): return 0
        return code[code[pointer+offset]]
    elif mode == 1: # immediate mode
        if pointer+offset >= len(code): return 0
        return code[pointer+offset]
    elif mode == 2: # relative mode
        if pointer+offset >= len(code): return 0 if relative+code[0] >= len(code) else code[relative]
        if relative+code[pointer+offset] >= len(code): return 0
        return code[relative+code[pointer+offset]]
    raise ValueError(f"{op}, {pointer}, {offset} not allowed")

class IO():
    # Simple implementation of IO: just has static input in the __init__

    def __init__(self, input_=None):
        self.input_buffer = input_ if input_ is not None else []
        self.output_buffer = []

    def read(self):
        drawer(self.output_buffer)
        time.sleep(0.05)
        self.output_buffer = []
        result = int(input("Your input? (-1=left, 0=stay, 1=right)"))
        clear_output(wait=True)
        time.sleep(0.05)
        return result
    
    def write(self, value):
        self.output_buffer.append(value)
        return self
    
    def output(self):
        return self.output_buffer

def step(code, p, r, io):
    # INPLACE does a intcode step, and returns new state and new p
    op = code[p]
    if op % 100 == 1: # add
        put(code, op, p, r, 3, get(op, code, p, r, 1) + get(op, code, p, r, 2))
        return code, p+4, r, io
    elif op % 100 == 2: # mul
        put(code, op, p, r, 3, get(op, code, p, r, 1) * get(op, code, p, r, 2))
        return code, p+4, r, io
    elif op % 100 == 3: # read
        put(code, op, p, r, 1, io.read())
        return code, p+2, r, io
    elif op % 100 == 4: # write
        return code, p+2, r, io.write(get(op, code, p, r, 1))
    elif op % 100 == 5: # jmp_f
        if get(op, code, p, r, 1) != 0:
            return code, get(op, code, p, r, 2), r, io
        return code, p+3, r, io
    elif op % 100 == 6: # jmp_t
        if get(op, code, p, r, 1) == 0:
            return code, get(op, code, p, r, 2), r, io
        return code, p+3, r, io
    elif op % 100 == 7: # lt
        put(code, op, p, r, 3, int(get(op, code, p, r, 1) < get(op, code, p, r, 2)))
        return code, p+4, r, io
    elif op % 100 == 8: # eq
        put(code, op, p, r, 3, int(get(op, code, p, r, 1) == get(op, code, p, r, 2)))
        return code, p+4, r, io
    elif op % 100 == 9: # relative
        return code, p+2, r + get(op, code, p, r, 1), io
    elif op % 100 == 99: # exit
        return code, -1, r, io
    raise ValueError(f"Incorrect program. Op is {op}")

def run(code, io):
    pointer = 0
    relative = 0
    while pointer >= 0:
        code, pointer, relative, io = step(code, pointer, relative, io)
    return code, io.output()


In [210]:
import advent
data = advent.get_intcode(13)

_, out = run(data.copy(), IO())
len([x for x in out[2::3] if x == 2])

341

In [211]:
drawer(out)

43 19 0 0
||||||||||||||||||||||||||||||||||||||||||||
|                                          |
| XXXX  XX X  XXXX XXXX X X X X X X X X  X |
| X  X XXX X XXXXXX XXXXXXX X X   XX  XXXX |
| XXXXXXXX XXXXXX XXXX   XX XXXXXX  XXXXXX |
| X XXXXX  X  X XXXXXXX X  X XXX XXXXXXXXX |
| XXX X  XX XXX X XXXXX   X  XXX XX   XXX  |
| XXXXXXX  XXXX  XXX XXX XXXX X XX XXX X X |
| XXXX X XXXXXXXX   XXX X X XXXX XX X X X  |
| XXXX XX XX   X XXXX X   XX XXXX  XXXX XX |
| XXX XX XXXXXXXXXXX X  XXXX XXX XXXXXX XX |
| XXXXX XXX XX  XXX XXXXXX X XX  XX XXX XX |
| XXXXX XXX XXXXX X XXX X  X XXXXXXX XXXXX |
|  XXXXXX XXXXXXX X XXXXXXXX X  XX X XXXXX |
|                                          |
|                   o                      |
|                                          |
|                                          |
|                     -                    |
|                                          |
Score: 0


In [212]:
code = data.copy()
code[0] = 2 # insert coin
_, out = run(code, IO())

43 19 -1 0
||||||||||||||||||||||||||||||||||||||||||||
                                          ||
 XXXX  XX X  XXXX XXXX X X X X X X X X  X ||
 X  X XXX X XXXXXX XXXXXXX X X   XX  XXXX ||
 XXXXXXXX XXXXXX XXXX   XX XXXXXX  XXXXXX ||
 X XXXXX  X  X XXXXXXX X  X XXX XXXXXXXXX ||
 XXX X  XX XXX X XXXXX   X  XXX XX   XXX  ||
 XXXXXXX  XXXX  XXX XXX XXXX X XX XXX X X ||
 XXXX X XXXXXXXX   XXX X X XXXX XX X X X  ||
 XXXX XX XX   X XXXX X   XX XXXX  XXXX XX ||
 XXX XX XXXXXXXXXXX X  XXXX XXX XXXXXX XX ||
 XXXXX XXX XX  XXX XXXXXX X XX  XX XXX XX ||
 XXXXX XXX XXXXX X XXX X  X XXXXXXX XXXXX ||
  XXXXXX XXXXXXX X XXXXXXXX X  XX X XXXXX ||
                                          ||
                   o                      ||
                                          ||
                                          ||
                     -                    ||
                                          ||
Score: 0


ValueError: invalid literal for int() with base 10: ''